In [1]:
using Base.Test
include("../src/CPDs/cpds.jl")
using .CPDs

In [2]:
# StaticCPD
let
    df = DataFrame(a=randn(100))
    cpd = fit(CPD{Normal, StaticCPD}, df, :a)
    @test name(cpd) == :a
    @test parentless(cpd)
    @test condition!(cpd, Assignment()) === distribution(cpd)
    @test pdf(cpd, Assignment(:a=>0.5)) > 0.2
    @test pdf(cpd, Assignment(:a=>0.0)) > pdf(cpd, Assignment(:a=>0.5))
end

In [11]:
# Linear Gaussian
let

    # no parents
    let
        df = DataFrame(a=randn(100))
        cpd = fit(CPD{Normal{Float64}, LinearGaussianCPD}, df, :a, min_stdev=0.0)

        @test name(cpd) == :a
        @test parentless(cpd)
        @test distribution(cpd) == condition!(cpd, Assignment())
        @test abs(distribution(cpd).σ - 1.0) < 0.2
        @test pdf(cpd, Dict{Symbol, Any}(:a=>0.5)) > 0.2
        @test pdf(cpd, Dict{Symbol, Any}(:a=>0.0)) > pdf(cpd, Dict{Symbol, Any}(:a=>0.5))
    end

    # with parents
    let
        a = randn(1000)
        b = randn(1000) .+ 2*a .+ 1

        df = DataFrame(a=a, b=b)
        cpd = fit(CPD{Normal{Float64}, LinearGaussianCPD}, df, :b, [:a])

        @test !parentless(cpd)

        p = condition!(cpd, Assignment(:a=>0.0))
        @test isapprox(p.μ, 1.0, atol=0.25)
        @test isapprox(p.σ, 2.0, atol=0.50)

        p = condition!(cpd, Assignment(:a=>1.0))
        @test isapprox(p.μ, 3.0, atol=0.25)
        @test isapprox(p.σ, 2.0, atol=0.50)

        cpd = fit(CPD{Normal{Float64}, LinearGaussianCPD}, df, :b, [:a], min_stdev=10.0)
        @test distribution(cpd).σ == 10.0
    end
end

In [6]:
# CategoricalCPD
let

    # no parents
    let
        df = DataFrame(a=[1,2,1,2,3])
        cpd = fit(CPD{Categorical, CategoricalCPD}, df, :a)

        @test name(cpd) == :a
        @test parentless(cpd)
        @test distribution(cpd) === condition!(cpd, Assignment())
        @test pdf(distribution(cpd), 1) == 0.4
        @test pdf(distribution(cpd), 2) == 0.4
        @test pdf(distribution(cpd), 3) == 0.2
    end

    # with parents
    let
        df = DataFrame(a=[1,2,1,2,3], b=[1,1,2,1,2])
        cpd = fit(CPD{Categorical, CategoricalCPD}, df, :b, [:a])

        @test condition!(cpd, Assignment(:a=>1)).p == [0.5,0.5]
        @test condition!(cpd, Assignment(:a=>2)).p == [1.0,0.0]
        @test condition!(cpd, Assignment(:a=>3)).p == [0.0,1.0]

        cpd = fit(CPD{Categorical, CategoricalCPD}, df, :b, [:a], dirichlet_prior=1.0)
        @test condition!(cpd, Assignment(:a=>1)).p == [0.50,0.50]
        @test condition!(cpd, Assignment(:a=>2)).p == [0.75,0.25]
        @test condition!(cpd, Assignment(:a=>3)).p == [ 1/3, 2/3]
    end
end

CPDs.LinearGaussianCPD(#undef,0.0)

# Test

In [5]:
a = randn(100)
b = randn(100) .+ 2*a .+ 1

df = DataFrame(a=a, b=b)
cpd = fit(LinearGaussianCPD, df, :b, [:a])

CPDs.LinearGaussianCPD(CPDs.CPDCore{Distributions.Normal{Float64}}(:b,[:a],Distributions.Normal{Float64}(μ=NaN, σ=2.1179207832752054)),[1.9933443971293257],1.0765022745202344)

In [6]:
distribution(cpd).σ

2.1179207832752054

In [7]:
condition!(cpd, Dict(:a=>1.0))

Distributions.Normal{Float64}(μ=3.06984667164956, σ=2.1179207832752054)

In [10]:
rand(cpd)

LoadError: LoadError: MethodError: `rand` has no method matching rand(::CPDs.LinearGaussianCPD)
Closest candidates are:
  rand(::CPDs.CPD{D<:Distributions.Distribution{F<:Distributions.VariateForm,S<:Distributions.ValueSupport}}, !Matched::Dict{K,V})
while loading In[10], in expression starting on line 1